In [1]:
import pandas as pd
import numpy as np
import torch
import torch.utils.data as Data
import matplotlib.pyplot as plt
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler

In [2]:
# read file
stock_path = "C:/Users/USER/Desktop/Lab/lab4_2.csv"
stock_df = pd.read_csv(stock_path)
stock_df = stock_df[1:]

In [3]:
stock_df

,Date,Open_Change(%),High_Change(%),Low_Change(%),Close_Change(%),K r/f,D r/f,MACD r/f,SMA r/f,RISE
1,2010/1/21,-0.166353,-2.097008,-4.047132,-6.219438,-1,-1,-1,-1,-1
2,2010/1/22,-5.666917,-0.658867,-0.527480,3.141455,-1,-1,-1,-1,1
3,2010/1/25,4.417183,-1.990007,-2.826685,-6.937428,-1,-1,-1,-1,-1
4,2010/1/26,-6.937428,-6.260734,-6.909228,-6.909228,-1,-1,-1,-1,-1
5,2010/1/27,-6.909228,-4.873456,-3.906296,0.000000,-1,-1,-1,-1,0
...,...,...,...,...,...,...,...,...,...,...
2434,2019/12/25,-0.701758,-0.863558,-0.530034,0.706717,1,1,1,1,1
2435,2019/12/26,2.473502,3.135887,2.309057,2.280700,1,1,1,1,1
2436,2019/12/27,1.034479,0.675671,1.388896,1.886798,1,1,1,1,1
2437,2019/12/30,5.460753,9.563767,5.821912,9.932661,1,1,1,1,1


In [4]:
scaler= MinMaxScaler(feature_range = (-1,1))

In [5]:
minmax_df = scaler.fit_transform(stock_df[["Open_Change(%)","High_Change(%)","Low_Change(%)","Close_Change(%)"]].values)

In [6]:
print(minmax_df)

[[-0.04530549 -0.09189919 -0.52636024 -0.62124465]
 [-0.4447902   0.04276637 -0.24415337  0.31657508]
 [ 0.28757901 -0.08187981 -0.42850435 -0.69317645]
 ...
 [ 0.04190633  0.16773068 -0.09049776  0.19087739]
 [ 0.36336948  1.          0.26494293  0.99695111]
 [ 0.37808966  0.41993612  0.14844263  0.33937774]]


In [7]:
minmax_df = pd.DataFrame(minmax_df,columns = ["Open_Change(%)","High_Change(%)","Low_Change(%)","Close_Change(%)"])

In [8]:
minmax_df["K r/f"] = stock_df["K r/f"].values
minmax_df["D r/f"] = stock_df["D r/f"].values
minmax_df["MACD r/f"] = stock_df["MACD r/f"].values
minmax_df["SMA r/f"] = stock_df["SMA r/f"].values
minmax_df["RISE"] = stock_df["RISE"].values
minmax_df["Date"] = stock_df["Date"].values


In [9]:
minmax_df[:10]

,Open_Change(%),High_Change(%),Low_Change(%),Close_Change(%),K r/f,D r/f,MACD r/f,SMA r/f,RISE,Date
0,-0.045305,-0.091899,-0.526360,-0.621245,-1,-1,-1,-1,-1,2010/1/21
1,-0.444790,0.042766,-0.244153,0.316575,-1,-1,-1,-1,1,2010/1/22
2,0.287579,-0.081880,-0.428504,-0.693176,-1,-1,-1,-1,-1,2010/1/25
3,-0.537063,-0.481785,-0.755844,-0.690351,-1,-1,-1,-1,-1,2010/1/26
4,-0.535014,-0.351882,-0.515068,0.001849,-1,-1,-1,-1,0,2010/1/27
5,0.179564,0.459829,0.254462,0.686732,1,-1,1,-1,1,2010/1/28
6,0.214816,0.549530,-0.016832,0.239940,1,-1,1,-1,1,2010/1/29
7,0.166680,-0.026281,-0.081281,-0.087620,1,1,1,-1,-1,2010/2/1
8,0.031609,0.104462,-0.454291,-0.557745,-1,-1,-1,-1,-1,2010/2/2
9,-0.431706,0.021611,-0.079206,0.690164,1,1,1,1,1,2010/2/3


In [10]:
len(minmax_df)

2438

In [11]:
train_set = minmax_df[:2200]
test_set = minmax_df[2200:]
print(train_set.shape)
print(test_set.shape)

(2200, 10)
(238, 10)


In [12]:
#hyper parameter
torch.manual_seed(1)
EPOCH = 100000
BATCH_SIZE = 32
TIME_STEP = 7
INPUT_SIZE = 8
LR = 0.01

In [13]:
# declear training features data
features = []
for i in range(TIME_STEP,len(minmax_df)):
    x = minmax_df[i-TIME_STEP:i][["Open_Change(%)","High_Change(%)","Low_Change(%)","Close_Change(%)","K r/f","D r/f","MACD r/f","SMA r/f"]].values
    features.append(x.tolist())
features = torch.FloatTensor(features)
print("size of feature: ",features.size())

# declear trainging labels data
labels = []
for i in range(TIME_STEP,len(minmax_df)):
    x = minmax_df[i:i+1]["RISE"]
    labels.append(x.tolist())
labels = torch.FloatTensor(labels)
print("size of label: ",labels.size())

size of feature:  torch.Size([2431, 7, 8])
size of label:  torch.Size([2431, 1])


In [14]:
# random select 90% training set index and 10% testing set index 

x = np.linspace(0,2430,2431).tolist()
for i in range(0,2431):
    x[i] = int(x[i])

x = np.random.shuffle(x)
training, test = x[:2200], x[2200:]
test.sort()
training.sort()

In [15]:
train_features = features[training]
train_labels = labels[training]
test_features = features[test]
test_labels = labels[test]

In [16]:

#Mini-Batch

torch_dataset = Data.TensorDataset(train_features,train_labels)
train_loader = Data.DataLoader(
    dataset = torch_dataset,
    batch_size = BATCH_SIZE,
    num_workers = 2,
    shuffle=True,
)

In [17]:

#define NN architecture


class RNN(torch.nn.Module):   
    def __init__(self):
        super(RNN,self).__init__()
        # define lstm layer
        self.lstm = torch.nn.LSTM(
            input_size=INPUT_SIZE,
            hidden_size=128,         
            num_layers=1,
            batch_first=True, 
        )
        self.lstm2 = torch.nn.LSTM(
            input_size=128,
            hidden_size=64,         
            num_layers=1,
            batch_first=True, 
        )
        # dropout layer
        self.Relu =  torch.nn.ReLU()
        self.dropout = torch.nn.Dropout(p = 0.5)
        self.out = torch.nn.Linear(64,1)
        
    def forward(self,x):
        lstm1_out,(h_n,h_c) = self.lstm(x,None)
        lstm1_out = self.dropout(lstm1_out)
        lstm2_out,_ = self.lstm2(lstm1_out,None)
        lstm2_out = self.Relu(lstm2_out[:,-1,:])
        out = self.out(lstm2_out)
        
        return out


In [18]:
model = RNN()
print(model)

RNN(
  (lstm): LSTM(8, 128, batch_first=True)
  (lstm2): LSTM(128, 64, batch_first=True)
  (Relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (out): Linear(in_features=64, out_features=1, bias=True)
)


In [19]:
# define optimizer and loss function 
optimizer = torch.optim.Adam(model.parameters(), lr=LR,weight_decay=0.001)
# adject learning rate . when loss don't fall , lr = lr * factor  , min lr = 0.0001
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,factor = 0.98,min_lr=0.0001)
# crossentroy loss 
loss_func = torch.nn.MSELoss()

In [ ]:
'''
Training 
'''
LOSS = []
TEST_LOSS = []
TEST_ACC = []
TRAIN_ACC = []
for epoch in range(EPOCH):
    loss_total = 0
    for step,(inputs,targets) in enumerate(train_loader):
        inputs = inputs.view(-1,TIME_STEP,INPUT_SIZE)
        # start trainnig 
        output = model(inputs)
        # calculate loss  (cross entroy)
        loss = loss_func(output,targets)
        # clear the gradients of all optimized variables(from last training)
        optimizer.zero_grad()
        # back propagation
       
        loss.backward()
        optimizer.step()
        # sum of loss
        loss_total = loss_total + loss
        
        # print training info every 30 steps
        if((step+1) %30 == 0):
            # average of loss in 30 steps
            avg = loss_total / 30
            LOSS.append(avg.tolist())
            
            # calculate the accuracy of training 
            '''
            code
            '''
            
            # calculate the accuracy of using testing data as inputs
            '''
            code
            '''
            test_output = model(test_features.view(-1,TIME_STEP,INPUT_SIZE))
            test_loss = loss_func(test_output,test_labels)
            TEST_LOSS.append(test_loss.tolist())
            # print the epoch , steps , average loss , accuracy 
            print("Epoch: %4d|steps: %4d|Train Avg Loss: %.4f |Test Loss: %.4f|lr = %.5f"
                  %(epoch+1,step+1,avg,test_loss,optimizer.param_groups[0]['lr']))
            
            # inital variable
            loss_total = 0
    # updata learning rate
    scheduler.step(loss)
        


Epoch:    1|steps:   30|Train Avg Loss: 0.9315 |Test Loss: 0.9315|lr = 0.01000
Epoch:    1|steps:   60|Train Avg Loss: 0.9373 |Test Loss: 0.9774|lr = 0.01000
Epoch:    2|steps:   30|Train Avg Loss: 0.9343 |Test Loss: 0.9539|lr = 0.01000
Epoch:    2|steps:   60|Train Avg Loss: 0.9295 |Test Loss: 0.9339|lr = 0.01000
Epoch:    3|steps:   30|Train Avg Loss: 0.9326 |Test Loss: 0.9365|lr = 0.01000
Epoch:    3|steps:   60|Train Avg Loss: 0.9358 |Test Loss: 0.9730|lr = 0.01000
Epoch:    4|steps:   30|Train Avg Loss: 0.9250 |Test Loss: 0.9691|lr = 0.01000
Epoch:    4|steps:   60|Train Avg Loss: 0.9354 |Test Loss: 0.9452|lr = 0.01000
Epoch:    5|steps:   30|Train Avg Loss: 0.9344 |Test Loss: 0.9637|lr = 0.01000
Epoch:    5|steps:   60|Train Avg Loss: 0.9292 |Test Loss: 0.9442|lr = 0.01000
Epoch:    6|steps:   30|Train Avg Loss: 0.9279 |Test Loss: 0.9620|lr = 0.01000
Epoch:    6|steps:   60|Train Avg Loss: 0.9412 |Test Loss: 0.9757|lr = 0.01000
Epoch:    7|steps:   30|Train Avg Loss: 0.9285 |Test

Epoch:   53|steps:   30|Train Avg Loss: 0.9226 |Test Loss: 0.9641|lr = 0.00922
Epoch:   53|steps:   60|Train Avg Loss: 0.9402 |Test Loss: 0.9493|lr = 0.00922
Epoch:   54|steps:   30|Train Avg Loss: 0.9300 |Test Loss: 0.9500|lr = 0.00922
Epoch:   54|steps:   60|Train Avg Loss: 0.9324 |Test Loss: 0.9832|lr = 0.00922
Epoch:   55|steps:   30|Train Avg Loss: 0.9238 |Test Loss: 0.9586|lr = 0.00922
Epoch:   55|steps:   60|Train Avg Loss: 0.9356 |Test Loss: 0.9568|lr = 0.00922
Epoch:   56|steps:   30|Train Avg Loss: 0.9348 |Test Loss: 0.9574|lr = 0.00922
Epoch:   56|steps:   60|Train Avg Loss: 0.9423 |Test Loss: 0.9650|lr = 0.00922
Epoch:   57|steps:   30|Train Avg Loss: 0.9373 |Test Loss: 0.9462|lr = 0.00922
Epoch:   57|steps:   60|Train Avg Loss: 0.9315 |Test Loss: 0.9774|lr = 0.00922
Epoch:   58|steps:   30|Train Avg Loss: 0.9416 |Test Loss: 0.9485|lr = 0.00922
Epoch:   58|steps:   60|Train Avg Loss: 0.9237 |Test Loss: 1.0302|lr = 0.00922
Epoch:   59|steps:   30|Train Avg Loss: 0.9367 |Test

Epoch:  105|steps:   30|Train Avg Loss: 0.9285 |Test Loss: 0.9688|lr = 0.00851
Epoch:  105|steps:   60|Train Avg Loss: 0.9340 |Test Loss: 0.9495|lr = 0.00851
Epoch:  106|steps:   30|Train Avg Loss: 0.9318 |Test Loss: 0.9511|lr = 0.00851
Epoch:  106|steps:   60|Train Avg Loss: 0.9327 |Test Loss: 0.9613|lr = 0.00851
Epoch:  107|steps:   30|Train Avg Loss: 0.9303 |Test Loss: 0.9578|lr = 0.00834
Epoch:  107|steps:   60|Train Avg Loss: 0.9272 |Test Loss: 0.9622|lr = 0.00834
Epoch:  108|steps:   30|Train Avg Loss: 0.9435 |Test Loss: 0.9545|lr = 0.00834
Epoch:  108|steps:   60|Train Avg Loss: 0.9244 |Test Loss: 0.9460|lr = 0.00834
Epoch:  109|steps:   30|Train Avg Loss: 0.9337 |Test Loss: 0.9652|lr = 0.00834
Epoch:  109|steps:   60|Train Avg Loss: 0.9339 |Test Loss: 0.9584|lr = 0.00834
Epoch:  110|steps:   30|Train Avg Loss: 0.9347 |Test Loss: 0.9632|lr = 0.00834
Epoch:  110|steps:   60|Train Avg Loss: 0.9246 |Test Loss: 0.9545|lr = 0.00834
Epoch:  111|steps:   30|Train Avg Loss: 0.9223 |Test

Epoch:  157|steps:   30|Train Avg Loss: 0.9445 |Test Loss: 0.9555|lr = 0.00769
Epoch:  157|steps:   60|Train Avg Loss: 0.9293 |Test Loss: 0.9529|lr = 0.00769
Epoch:  158|steps:   30|Train Avg Loss: 0.9308 |Test Loss: 0.9497|lr = 0.00769
Epoch:  158|steps:   60|Train Avg Loss: 0.9378 |Test Loss: 0.9570|lr = 0.00769
Epoch:  159|steps:   30|Train Avg Loss: 0.9245 |Test Loss: 0.9599|lr = 0.00769
Epoch:  159|steps:   60|Train Avg Loss: 0.9337 |Test Loss: 0.9506|lr = 0.00769
Epoch:  160|steps:   30|Train Avg Loss: 0.9262 |Test Loss: 0.9501|lr = 0.00769
Epoch:  160|steps:   60|Train Avg Loss: 0.9462 |Test Loss: 0.9336|lr = 0.00769
Epoch:  161|steps:   30|Train Avg Loss: 0.9463 |Test Loss: 0.9923|lr = 0.00769
Epoch:  161|steps:   60|Train Avg Loss: 0.9276 |Test Loss: 0.9525|lr = 0.00769
Epoch:  162|steps:   30|Train Avg Loss: 0.9317 |Test Loss: 0.9703|lr = 0.00754
Epoch:  162|steps:   60|Train Avg Loss: 0.9364 |Test Loss: 0.9468|lr = 0.00754
Epoch:  163|steps:   30|Train Avg Loss: 0.9359 |Test

Epoch:  209|steps:   30|Train Avg Loss: 0.9230 |Test Loss: 0.9766|lr = 0.00695
Epoch:  209|steps:   60|Train Avg Loss: 0.9402 |Test Loss: 0.9613|lr = 0.00695
Epoch:  210|steps:   30|Train Avg Loss: 0.9268 |Test Loss: 0.9712|lr = 0.00695
Epoch:  210|steps:   60|Train Avg Loss: 0.9303 |Test Loss: 0.9559|lr = 0.00695
Epoch:  211|steps:   30|Train Avg Loss: 0.9157 |Test Loss: 0.9751|lr = 0.00695
Epoch:  211|steps:   60|Train Avg Loss: 0.9357 |Test Loss: 0.9517|lr = 0.00695
Epoch:  212|steps:   30|Train Avg Loss: 0.9262 |Test Loss: 0.9631|lr = 0.00695
Epoch:  212|steps:   60|Train Avg Loss: 0.9374 |Test Loss: 0.9606|lr = 0.00695
Epoch:  213|steps:   30|Train Avg Loss: 0.9271 |Test Loss: 0.9520|lr = 0.00695
Epoch:  213|steps:   60|Train Avg Loss: 0.9406 |Test Loss: 0.9566|lr = 0.00695
Epoch:  214|steps:   30|Train Avg Loss: 0.9433 |Test Loss: 0.9415|lr = 0.00695
Epoch:  214|steps:   60|Train Avg Loss: 0.9278 |Test Loss: 0.9990|lr = 0.00695
Epoch:  215|steps:   30|Train Avg Loss: 0.9312 |Test

...

Epoch: 8425|steps:   30|Train Avg Loss: 0.0034 |Test Loss: 1.7625|lr = 0.00010
Epoch: 8425|steps:   60|Train Avg Loss: 0.0038 |Test Loss: 1.7762|lr = 0.00010
Epoch: 8426|steps:   30|Train Avg Loss: 0.0039 |Test Loss: 1.7729|lr = 0.00010
Epoch: 8426|steps:   60|Train Avg Loss: 0.0032 |Test Loss: 1.7740|lr = 0.00010
Epoch: 8427|steps:   30|Train Avg Loss: 0.0033 |Test Loss: 1.7720|lr = 0.00010
Epoch: 8427|steps:   60|Train Avg Loss: 0.0029 |Test Loss: 1.7779|lr = 0.00010
Epoch: 8428|steps:   30|Train Avg Loss: 0.0015 |Test Loss: 1.7526|lr = 0.00010
Epoch: 8428|steps:   60|Train Avg Loss: 0.0028 |Test Loss: 1.7552|lr = 0.00010
Epoch: 8429|steps:   30|Train Avg Loss: 0.0029 |Test Loss: 1.7691|lr = 0.00010
Epoch: 8429|steps:   60|Train Avg Loss: 0.0024 |Test Loss: 1.7707|lr = 0.00010
Epoch: 8430|steps:   30|Train Avg Loss: 0.0025 |Test Loss: 1.7656|lr = 0.00010
Epoch: 8430|steps:   60|Train Avg Loss: 0.0024 |Test Loss: 1.7588|lr = 0.00010
Epoch: 8431|steps:   30|Train Avg Loss: 0.0022 |Test

Epoch: 8477|steps:   30|Train Avg Loss: 0.0027 |Test Loss: 1.7698|lr = 0.00010
Epoch: 8477|steps:   60|Train Avg Loss: 0.0033 |Test Loss: 1.7700|lr = 0.00010
Epoch: 8478|steps:   30|Train Avg Loss: 0.0025 |Test Loss: 1.7697|lr = 0.00010
Epoch: 8478|steps:   60|Train Avg Loss: 0.0028 |Test Loss: 1.7807|lr = 0.00010
Epoch: 8479|steps:   30|Train Avg Loss: 0.0024 |Test Loss: 1.7612|lr = 0.00010
Epoch: 8479|steps:   60|Train Avg Loss: 0.0029 |Test Loss: 1.7739|lr = 0.00010
Epoch: 8480|steps:   30|Train Avg Loss: 0.0033 |Test Loss: 1.7788|lr = 0.00010
Epoch: 8480|steps:   60|Train Avg Loss: 0.0026 |Test Loss: 1.7721|lr = 0.00010
Epoch: 8481|steps:   30|Train Avg Loss: 0.0022 |Test Loss: 1.7508|lr = 0.00010
Epoch: 8481|steps:   60|Train Avg Loss: 0.0023 |Test Loss: 1.7607|lr = 0.00010
Epoch: 8482|steps:   30|Train Avg Loss: 0.0032 |Test Loss: 1.7554|lr = 0.00010
Epoch: 8482|steps:   60|Train Avg Loss: 0.0019 |Test Loss: 1.7756|lr = 0.00010
Epoch: 8483|steps:   30|Train Avg Loss: 0.0027 |Test

Epoch: 8529|steps:   30|Train Avg Loss: 0.0022 |Test Loss: 1.7849|lr = 0.00010
Epoch: 8529|steps:   60|Train Avg Loss: 0.0021 |Test Loss: 1.7794|lr = 0.00010
Epoch: 8530|steps:   30|Train Avg Loss: 0.0019 |Test Loss: 1.7511|lr = 0.00010
Epoch: 8530|steps:   60|Train Avg Loss: 0.0022 |Test Loss: 1.7682|lr = 0.00010
Epoch: 8531|steps:   30|Train Avg Loss: 0.0020 |Test Loss: 1.7722|lr = 0.00010
Epoch: 8531|steps:   60|Train Avg Loss: 0.0027 |Test Loss: 1.7741|lr = 0.00010
Epoch: 8532|steps:   30|Train Avg Loss: 0.0023 |Test Loss: 1.7589|lr = 0.00010
Epoch: 8532|steps:   60|Train Avg Loss: 0.0016 |Test Loss: 1.7735|lr = 0.00010
Epoch: 8533|steps:   30|Train Avg Loss: 0.0027 |Test Loss: 1.7560|lr = 0.00010
Epoch: 8533|steps:   60|Train Avg Loss: 0.0022 |Test Loss: 1.7698|lr = 0.00010
Epoch: 8534|steps:   30|Train Avg Loss: 0.0025 |Test Loss: 1.7509|lr = 0.00010
Epoch: 8534|steps:   60|Train Avg Loss: 0.0027 |Test Loss: 1.7672|lr = 0.00010
Epoch: 8535|steps:   30|Train Avg Loss: 0.0030 |Test

Epoch: 8581|steps:   30|Train Avg Loss: 0.0021 |Test Loss: 1.7762|lr = 0.00010
Epoch: 8581|steps:   60|Train Avg Loss: 0.0029 |Test Loss: 1.7671|lr = 0.00010
Epoch: 8582|steps:   30|Train Avg Loss: 0.0034 |Test Loss: 1.7652|lr = 0.00010
Epoch: 8582|steps:   60|Train Avg Loss: 0.0028 |Test Loss: 1.7817|lr = 0.00010
Epoch: 8583|steps:   30|Train Avg Loss: 0.0023 |Test Loss: 1.7705|lr = 0.00010
Epoch: 8583|steps:   60|Train Avg Loss: 0.0030 |Test Loss: 1.7657|lr = 0.00010
Epoch: 8584|steps:   30|Train Avg Loss: 0.0039 |Test Loss: 1.7976|lr = 0.00010
Epoch: 8584|steps:   60|Train Avg Loss: 0.0038 |Test Loss: 1.7901|lr = 0.00010
Epoch: 8585|steps:   30|Train Avg Loss: 0.0024 |Test Loss: 1.7785|lr = 0.00010
Epoch: 8585|steps:   60|Train Avg Loss: 0.0030 |Test Loss: 1.7570|lr = 0.00010
Epoch: 8586|steps:   30|Train Avg Loss: 0.0020 |Test Loss: 1.7698|lr = 0.00010
Epoch: 8586|steps:   60|Train Avg Loss: 0.0023 |Test Loss: 1.7748|lr = 0.00010
Epoch: 8587|steps:   30|Train Avg Loss: 0.0023 |Test

Epoch: 8633|steps:   30|Train Avg Loss: 0.0026 |Test Loss: 1.7745|lr = 0.00010
Epoch: 8633|steps:   60|Train Avg Loss: 0.0020 |Test Loss: 1.7448|lr = 0.00010
Epoch: 8634|steps:   30|Train Avg Loss: 0.0021 |Test Loss: 1.7484|lr = 0.00010
Epoch: 8634|steps:   60|Train Avg Loss: 0.0027 |Test Loss: 1.7610|lr = 0.00010
Epoch: 8635|steps:   30|Train Avg Loss: 0.0022 |Test Loss: 1.7583|lr = 0.00010
Epoch: 8635|steps:   60|Train Avg Loss: 0.0025 |Test Loss: 1.7244|lr = 0.00010
Epoch: 8636|steps:   30|Train Avg Loss: 0.0039 |Test Loss: 1.7843|lr = 0.00010
Epoch: 8636|steps:   60|Train Avg Loss: 0.0031 |Test Loss: 1.7539|lr = 0.00010
Epoch: 8637|steps:   30|Train Avg Loss: 0.0022 |Test Loss: 1.7605|lr = 0.00010
Epoch: 8637|steps:   60|Train Avg Loss: 0.0031 |Test Loss: 1.7721|lr = 0.00010
Epoch: 8638|steps:   30|Train Avg Loss: 0.0032 |Test Loss: 1.7587|lr = 0.00010
Epoch: 8638|steps:   60|Train Avg Loss: 0.0031 |Test Loss: 1.7688|lr = 0.00010
Epoch: 8639|steps:   30|Train Avg Loss: 0.0035 |Test

test = model(test_features.view(-1,TIME_STEP,INPUT_SIZE))
test = test.detach().numpy()
test = scaler.inverse_transform(test)

len(test)

stock_df

x = np.linspace(1,231,231)
y1 = stock_df[2208:2439]["Close"].values
y2 = test

plt.plot(x,y1)
plt.plot(x,y2)
plt.xlabel("Days")
plt.ylabel("Price")
plt.legend(labels=['Real', 'Predicted'],  loc='best')
plt.show()

x = np.linspace(1,200,200)
y1 = np.array(LOSS)
y2 = np.array(TEST_LOSS)
plt.plot(x,y1)
plt.plot(x,y2)
plt.xlabel("Iteration")
plt.ylabel("loss")
plt.legend(labels=['Train loss', 'Test loss'],  loc='best')
plt.show()

y1

stock_df.head(10)

t =stock_df[2208:2439]
t["predict"] = test

t.to_csv("C:/Users/acer/Desktop/LAB/123.csv")